This example is based on usage of free services such as GoogleColab and GoogleDrive. Make sure that we have access to the video card and google drive is connected:

In [1]:
!nvidia-smi

Thu May 14 06:37:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# ERNIE pretraining

## Knowledge embeddings

In [0]:
!python drive/'My Drive'/OpenKE/train_transe_wikidata.py \
        --benchmark drive/'My Drive'/OpenKE/benchmarks/wikidata/ \
        --checkpoint drive/'My Drive'/OpenKE/checkpoint/ \
        --epochs 1 \
        --alpha 0.001

In [0]:
!python drive/'My Drive'/OpenKE/get_embeddings.py \
        --benchmark drive/'My Drive'/OpenKE/benchmarks/wikidata/ \
        --checkpoint drive/'My Drive'/OpenKE/checkpoint/ \
        --embedding drive/'My Drive'/OpenKE/kg_embed/ \

Now you need to move that embeddings to ERNIE/kb_embed

In [20]:
!mv drive/'My Drive'/OpenKE/kg_embed/entity2vec.vec drive/'My Drive'/ERNIE/kg_embed/entity2vec.vec
!mv drive/'My Drive'/OpenKE/kg_embed/relation2vec.vec drive/'My Drive'/ERNIE/kg_embed/relation2vec.vec
!cp drive/'My Drive'/OpenKE/benchmarks/wikidata/entity2id.txt drive/'My Drive'/ERNIE/kb_embed/entity2id.txt

mv: cannot move 'drive/My Drive/OpenKE/kb_embed/entity2vec.vec' to 'drive/My Drive/ERNIE/kb_embed/entity2vec.vec': No such file or directory


And add "enity map.txt" from wikidata preprocessing to "ERNIE/kb_embed"

## Pretraining

Apex install example (for using fp16 to speed up training)

In [0]:
!pip show apex

In [0]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [0]:
%time
!sh setup.sh

ERNIE pretraining

In [21]:
!python drive/'My Drive'/ERNIE/code/run_pretrain.py

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
usage: run_pretrain.py [-h] --data_dir DATA_DIR --bert_model BERT_MODEL
                       --task_name TASK_NAME --output_dir OUTPUT_DIR
                       [--max_seq_length MAX_SEQ_LENGTH] [--do_train]
                       [--do_eval] [--do_lower_case]
                       [--train_batch_size TRAIN_BATCH_SIZE]
                       [--eval_batch_size EVAL_BATCH_SIZE]
                       [--learning_rate LEARNING_RATE]
                       [--num_train_epochs NUM_TRAIN_EPOCHS]
                       [--warmup_proportion WARMUP_PROPORTION] [--no_cuda]
                       [--local_rank LOCAL_RANK] [--seed SEED]
                       [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                       [--fp16] [--loss_scale LOSS_SCALE]
run_pretrain.py: error: the following arguments are required: --data_dir, --bert_model, --task_name, --output_dir


In [0]:
!python drive/'My Drive'/ERNIE/code/run_pretrain.py \
    --do_train \
    --data_dir drive/'My Drive'/ERNIE/pretrain_data/merge \
    --bert_model drive/'My Drive'/ERNIE/ernie_base \
    --output_dir drive/'My Drive'/ERNIE/pretrain_out/ \
    --kg_embed_dir drive/'My Drive'/ERNIE/kg_embed/ \
    --task_name pretrain \
    --max_seq_length 256 \
    --train_batch_size 8 \
    --num_train_epochs 1 \
    --steps_to_save 10

Словарь в ерни - левый, нужно поменять
Дописать авто чистку аутпута

Без записи в аут - быстрее в несколько раз по началу

Чтобы вернуть лимит GPU нужно создать новый ноутбук



# ERNIE fine-tuning

In progress